In [1]:
import pickle
import random

import numpy as np

matches_df = pickle.load(open("database/matches_df.pck", "rb"))

In [2]:
winner_list = []
market_val_diff = []
goals_diff = []
for _, row in matches_df.iterrows():
    if row["Team Home ID"] == row["Winner Team ID"]:
        winner_list.append("Home")
    elif row["Team Away ID"] == row["Winner Team ID"]:
        winner_list.append("Away")
    elif row["Winner Team ID"] == 0:
        winner_list.append("Draw")
    market_val_diff.append(row["Market Value Home"] - row["Market Value Away"])
    goals_diff.append(row["Goals Home"] - row["Goals Away"])
matches_df["Result"] = winner_list
matches_df["Market Value Difference"] = market_val_diff
matches_df["Goals Difference"] = goals_diff

In [3]:
import asyncio
import nest_asyncio
from database import dbconn
import importlib

importlib.reload(dbconn)

nest_asyncio.apply()

loop = asyncio.get_event_loop()



In [4]:
import pandas as pd

dict_team_data = dict()

for team_id in set(matches_df["Team Home ID"]):
    team_data_matches_home, team_data_matches_away = loop.run_until_complete(dbconn.get_matches_by_team(team_id))
    # Switched goalsAway and goalsHome to concat home and away matches to gotten and scored goals
    team_data_matches = [[x.id, x.goalsHome, x.goalsAway, x.winnerTeamId] for x in team_data_matches_home] + [[x.id, x.goalsAway, x.goalsHome, x.winnerTeamId] for x in team_data_matches_away]
    team_data_matches = sorted(team_data_matches, key=lambda x: x[0])
    for i in range(len(team_data_matches)):
        if team_data_matches[i][3] == team_id:
            team_data_matches[i][3] = "Won"
        elif team_data_matches[i][3] == None:
            team_data_matches[i][3] = "Draw"
        else:
            team_data_matches[i][3] = "Lost"
    dict_team_data[team_id] = pd.DataFrame(team_data_matches)

In [5]:
matches_form = []
for _, row in matches_df.iterrows():
    home_form = dict_team_data[row["Team Home ID"]]
    home_form = home_form[home_form[0] < row["Match ID"]].iloc[-5:]
    home_form_5 = list(home_form[3])
    for i in range(5 - len(home_form_5)):
        home_form_5 = ["Draw"] + home_form_5
    away_form = dict_team_data[row["Team Away ID"]]
    away_form = away_form[away_form[0] < row["Match ID"]].iloc[-5:]
    away_form_5 = list(away_form[3])
    for i in range(5 - len(away_form_5)):
        away_form_5 = ["Draw"] + away_form_5
    sum = 0
    home_form_5_sums = [0, 0, 0, 0, 0]
    for i in range(len(home_form_5)-1, -1, -1):
        if home_form_5[i] == "Draw":
            sum += 1
        elif home_form_5[i] == "Won":
            sum += 3
        home_form_5_sums[i] = sum
    sum = 0
    away_form_5_sums = [0, 0, 0, 0, 0]
    for i in range(len(away_form_5)-1, -1, -1):
        if away_form_5[i] == "Draw":
            sum += 1
        elif away_form_5[i] == "Won":
            sum += 3
        away_form_5_sums[i] = sum
    diff_form_5 = []
    for i in range(len(home_form_5)):
        diff_form_5.append(home_form_5_sums[i] - away_form_5_sums[i])
    matches_form.append(home_form_5 + away_form_5 + diff_form_5)

In [6]:
columns_added = ["Home_Pre" + str(x) for x in range(5, 0, -1)] + ["Away_Pre" + str(x) for x in range(5, 0, -1)] + ["Form_Diff_Pre" + str(x) for x in range(5, 0, -1)]
matches_df = pd.concat([matches_df, pd.DataFrame(matches_form, columns=columns_added)], axis=1)

In [7]:
matches_goals = []
for _, row in matches_df.iterrows():
    home_goals = dict_team_data[row["Team Home ID"]]
    home_goals = home_goals[home_goals[0] < row["Match ID"]].iloc[-5:]
    home_goals_5 = list(home_goals[1])
    for i in range(5 - len(home_goals_5)):
        home_goals_5 = [0] + home_goals_5
    away_goals = dict_team_data[row["Team Away ID"]]
    away_goals = away_goals[away_goals[0] < row["Match ID"]].iloc[-5:]
    away_goals_5 = list(away_goals[1])
    for i in range(5 - len(away_goals_5)):
        away_goals_5 = [0] + away_goals_5
    sum = 0
    home_goals_5_sums = [0, 0, 0, 0, 0]
    for i in range(len(home_goals_5)-1, -1, -1):
        sum += home_goals_5[i]
        home_goals_5_sums[i] = sum
    sum = 0
    away_goals_5_sums = [0, 0, 0, 0, 0]
    for i in range(len(away_goals_5)-1, -1, -1):
        sum += away_goals_5[i]
        away_goals_5_sums[i] = sum
    diff_goals_5 = []
    for i in range(len(home_goals_5)):
        diff_goals_5.append(home_goals_5_sums[i] - away_goals_5_sums[i])
    matches_goals.append(home_goals_5 + away_goals_5 + diff_goals_5)

In [8]:
goal_columns_added = ["Home_Pre_Goals" + str(x) for x in range(5, 0, -1)] + ["Away_Pre_Goals" + str(x) for x in range(5, 0, -1)] + ["Form_Diff_Pre_Goals" + str(x) for x in range(5, 0, -1)]
matches_df = pd.concat([matches_df, pd.DataFrame(matches_goals, columns=goal_columns_added)], axis=1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

end_goals = []
form_enc = LabelEncoder()
form_enc.fit_transform(matches_df["Home_Pre1"])
for col in columns_added[:-5]:
    matches_df[col] = form_enc.transform(matches_df[col])
for _, row in matches_df.iterrows():
    end_goals.append(str(row["Goals Home"]) + ":" + str(row["Goals Away"]))
result_enc = LabelEncoder()
end_goals = result_enc.fit_transform(end_goals)
matches_df["Goals Result"] = end_goals

In [10]:
res_enc = LabelEncoder()
matches_df["Result"] = res_enc.fit_transform(matches_df["Result"])
X_train, X_test, y_train, y_test = train_test_split(matches_df.drop(["Market Value Home", "Market Value Away", "Goals Home", "Goals Away", "Goals Difference", "Result", "Winner Team ID", "Match ID", "Team Home ID", "Team Away ID", "Goals Result"], axis=1), matches_df["Goals Result"], random_state=42, test_size=0.2)

In [11]:
from sklearn.metrics import make_scorer

def score(y_true, y_pred, **kwargs):
    y_true = [x.split(":") for x in result_enc.inverse_transform(y_true)]
    y_pred = [x.split(":") for x in result_enc.inverse_transform(y_pred)]
    score_value = 0
    for true, pred in zip(y_true, y_pred):
        if true[0] == pred[0] and true[1] == pred[1]:
            score_value += 5
        elif (int(true[0]) - int(true[1])) == (int(pred[0]) - int(pred[1])):
            score_value += 3
        elif ((true[0] > true[1]) and (pred[0] > pred[1])) or ((true[0] < true[1]) and (pred[0] < pred[1])):
            score_value += 1
    return round(score_value / (len(y_true)/306))

kicktipp_scorer = make_scorer(score, greater_is_better=True)

In [12]:
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

In [29]:
%%time

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

pipe = Pipeline(steps=[("clf", KNeighborsClassifier())])

param_grid = {
    "clf__n_neighbors": np.arange(10, 250, 5),
    "clf__weights": ["uniform", "distance"],
    "clf__metric": ["euclidean", "minkowski", "chebyshev", "cosine"]
}

clf = GridSearchCV(pipe, param_grid, n_jobs=-1, cv=tscv).fit(X_train, y_train)
y_pred = clf.predict(X_test)
score(y_test, y_pred), accuracy_score(y_test, y_pred), clf.best_params_, clf.best_score_

CPU times: user 574 ms, sys: 173 ms, total: 747 ms
Wall time: 5.25 s


(332,
 0.12581699346405228,
 {'clf__metric': 'euclidean',
  'clf__n_neighbors': 245,
  'clf__weights': 'uniform'},
 0.12107843137254903)